In [1]:
import os
import logging
import random
import gc
import time
import cv2
import math
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import librosa

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

import timm


logging.basicConfig(level=logging.ERROR)

In [2]:
class CFG:
    
    seed = 42
    debug = True  
    apex = False
    print_freq = 100
    num_workers = 2
    KAGGLE_NOTEBOOK = False
    
    if KAGGLE_NOTEBOOK:
        OUTPUT_DIR = '/kaggle/working/'
        train_datadir = '/kaggle/input/birdclef-2025/train_audio'
        train_csv = '/kaggle/input/birdclef-2025/train.csv'
        test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes'
        submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv'
        taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'
        spectrogram_npy = '/kaggle/input/birdclef25-mel-spectrograms/birdclef2025_melspec_5sec_256_256.npy'
    else:
        OUTPUT_DIR = '../data/result/'
        train_datadir = '../data/raw/train_audio/'
        train_csv = '../data/raw/train.csv'
        test_soundscapes = '../data/raw/test_soundscapes/'
        submission_csv = '../data/raw/sample_submission.csv'
        taxonomy_csv = '../data/raw/taxonomy.csv'
        spectrogram_npy = '../data/processed//birdclef2025_melspec_5sec_256_256.npy'

 
    model_name = 'efficientnet_b0'  
    pretrained = True
    in_channels = 1

    LOAD_DATA = True  
    FS = 32000
    TARGET_DURATION = 5.0
    TARGET_SHAPE = (256, 256)
    
    N_FFT = 1024
    HOP_LENGTH = 512
    N_MELS = 128
    FMIN = 50
    FMAX = 14000
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    epochs = 10  
    batch_size = 32  
    criterion = 'BCEWithLogitsLoss'

    n_fold = 5
    selected_folds = [0, 1, 2, 3, 4]   

    optimizer = 'AdamW'
    lr = 5e-4 
    weight_decay = 1e-5
  
    scheduler = 'CosineAnnealingLR'
    min_lr = 1e-6
    T_max = epochs

    aug_prob = 0.5  
    mixup_alpha = 0.5  
    
    def update_debug_settings(self):
        if self.debug:
            self.epochs = 2
            self.selected_folds = [0]

cfg = CFG()

In [3]:
def set_seed(seed=42):
    """
    Set seed for reproducibility
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(cfg.seed)